#Importing Libraries

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Generating Data using sklearn

In [ ]:
# Generate synthetic dataset with 2 features and 2 classes
X, y = make_classification(n_samples=1000, n_features=2, n_classes=2, n_clusters_per_class=1, n_redundant=0, random_state=42)
n_features=2


def normalise(X):
    u = X.mean(axis=0)
    std = X.std(axis=0)

    return (X-u)/std
X=normalise(X)

def addExtraColumn(X):
    if X.shape[1] == n_features:
        ones = np.ones((X.shape[0],1))
        X = np.hstack((ones,X))

    return X

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train=addExtraColumn(X_train)
X_test=addExtraColumn(X_test)

#Creating Logit Model

In [ ]:
class LogisticRegressionCustom:
    def __init__(self, learning_rate=0.01, num_iterations=1000, regularization_param=0.01):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.regularization_param = regularization_param

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def _compute_cost(self, X, y):
        m = len(y)
        h = self._sigmoid(np.dot(X, self.theta))
        cost = (1/m) * np.sum((-y * np.log(h)) - ((1 - y) * np.log(1 - h)))
        regularization_term = (self.regularization_param / (2 * m)) * np.sum(np.square(self.theta[1:]))
        cost += regularization_term
        return cost

    def fit(self, X, y):
        m, n = X.shape
        self.theta = np.zeros(n)
        for i in range(self.num_iterations):
            z = np.dot(X, self.theta)
            h = self._sigmoid(z)
            gradient = (1/m) * np.dot(X.T, (h - y))
            gradient[1:] += (self.regularization_param / m) * self.theta[1:]
            self.theta -= self.learning_rate * gradient

    def predict(self, X):
        z = np.dot(X, self.theta)
        predictions=self._sigmoid(z)
        return np.round(predictions)

    def score(self, X, y):
        predictions = np.round(self.predict(X))
        accuracy = np.mean(predictions == y)
        return accuracy

#Objective Function


In [ ]:
def objective_function(params, X_train, y_train, X_test, y_test):
    learning_Rate = 10**params[0]
    model = LogisticRegressionCustom(learning_rate=learning_Rate, num_iterations=1000, regularization_param=0.01)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy

#Grey Wolf Optimization Algorithm

In [ ]:
# Grey Wolf Optimization Algorithm
def grey_wolf_optimization(objective_function, dim=1, n_iter=100):
    num_wolves = 5
    lb, ub = -2, 2  # Lower and upper bounds for parameters (adjust according to your problem)
    alpha, beta, delta = np.random.uniform(lb, ub, (num_wolves, dim)), np.random.uniform(lb, ub, (num_wolves, dim)), np.random.uniform(lb, ub, (num_wolves, dim))

    for _ in range(n_iter):
        a, b, d = 2 - 2 * _ / n_iter, 2 * _ / n_iter, 2 * _ / n_iter
        for j in range(num_wolves):
            X1, X2, X3 = alpha[j].copy(), beta[j].copy(), delta[j].copy()
            for k in range(dim):
                X1[k] = alpha[j][k] - a * np.abs(2 * np.random.rand() * alpha[j][k] - X1[k])
                X2[k] = beta[j][k] - b * np.abs(2 * np.random.rand() * beta[j][k] - X2[k])
                X3[k] = delta[j][k] - d * np.abs(2 * np.random.rand() * delta[j][k] - X3[k])
            X_mean = (X1 + X2 + X3) / 3
            if objective_function(X_mean, X_train, y_train, X_test, y_test) < objective_function(alpha[j], X_train, y_train, X_test, y_test):
                alpha[j] = X_mean.copy()
            elif objective_function(alpha[j], X_train, y_train, X_test, y_test) < objective_function(X_mean, X_train, y_train, X_test, y_test) < objective_function(beta[j], X_train, y_train, X_test, y_test):
                beta[j] = X_mean.copy()
            elif objective_function(beta[j], X_train, y_train, X_test, y_test) < objective_function(X_mean, X_train, y_train, X_test, y_test) < objective_function(delta[j], X_train, y_train, X_test, y_test):
                delta[j] = X_mean.copy()

    best_wolf = alpha[np.argmin([objective_function(alpha[i], X_train, y_train, X_test, y_test) for i in range(num_wolves)])]
    return best_wolf



#Horse Herd Optimization Algorithm

In [ ]:
# Horse Herd Optimization Algorithm
def horse_herd_optimization(objective_function, dim=1, n_iter=100):
    num_horses = 5
    lb, ub = -2, 2  # Lower and upper bounds for parameters (adjust according to your problem)
    herd = np.random.uniform(lb, ub, (num_horses, dim))
    for _ in range(n_iter):
        for i in range(num_horses):
            rand1, rand2 = np.random.rand(), np.random.rand()
            A = 2 * rand1 - 1
            r1, r2 = np.random.rand(), np.random.rand()
            D = r1 * (herd[np.random.randint(num_horses)] - r2 * herd[np.random.randint(num_horses)])
            herd[i] = np.clip(A * herd[i] + D, lb, ub)

    best_horse = herd[np.argmin([objective_function(herd[i], X_train, y_train, X_test, y_test) for i in range(num_horses)])]
    return best_horse

#Evaluating model

In [ ]:
# Simple Logistic Regression Model
simple_lr =LogisticRegressionCustom(learning_rate=0.01, num_iterations=50, regularization_param=0.01)
simple_lr.fit(X_train, y_train)
simple_lr_accuracy = simple_lr.score(X_test, y_test)

# Logistic Regression Model with Grey Wolf Optimization
gwo_best_params = grey_wolf_optimization(objective_function, dim=1, n_iter=100)
gwo_lr = LogisticRegressionCustom(learning_rate=10**gwo_best_params[0], num_iterations=50, regularization_param=0.01)
gwo_lr.fit(X_train, y_train)
gwo_accuracy = gwo_lr.score(X_test, y_test)

# Logistic Regression Model with Horse Herd Optimization
hho_best_params = horse_herd_optimization(objective_function, dim=1, n_iter=100)
hho_lr = LogisticRegressionCustom(learning_rate=10**hho_best_params[0], num_iterations=100, regularization_param=0.01)
hho_lr.fit(X_train, y_train)
hho_accuracy = hho_lr.score(X_test, y_test)



#Presenting Results

In [ ]:
# Print the comparison results
print("Simple Logistic Regression Accuracy: {:.2f}%".format(simple_lr_accuracy * 100))
print("Logistic Regression with GWO Accuracy: {:.2f}%".format(gwo_accuracy * 100))
print("Logistic Regression with HHO Accuracy: {:.2f}%".format(hho_accuracy * 100))

Simple Logistic Regression Accuracy: 89.00%
Logistic Regression with GWO Accuracy: 89.50%
Logistic Regression with HHO Accuracy: 90.00%
